# Disclaimer

Since this exercise is comparatively extensive you will have **two weeks** to complete it. This means you have to hand it in until 30.05. at 10 am. Furthermore you are allowed to hand it in in groups of **two**! 

If you work together, please both hand in the Exercise sheet on the moodle page. 

# Datastructure of a triangulation

In this exercise you will implement the data structure discussed in [II.2](https://www.maths.ed.ac.uk/~v1ranick/papers/edelcomp.pdf). 

You will then use it to check if a triangulation is orientable.

In this exercise you will work with `StaticArrays`. A popular package for highly performant arrays in Julia. We are also including the `BenchmarkTools` package just in case you want to compare some running times. 🥇🥈

In [1]:
using Pkg
Pkg.activate("../CompTop2022")
Pkg.add("BenchmarkTools")
Pkg.add("StaticArrays")
using BenchmarkTools, StaticArrays

  Activating project at `~/Desktop/Doktor/Lehre/Exercise4/CompTop2022`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Doktor/Lehre/Exercise4/CompTop2022/Project.toml`
  No Changes to `~/Desktop/Doktor/Lehre/Exercise4/CompTop2022/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Doktor/Lehre/Exercise4/CompTop2022/Project.toml`
  No Changes to `~/Desktop/Doktor/Lehre/Exercise4/CompTop2022/Manifest.toml`


Let us begin by defining the `struct` that will represent a node in our graph. 

In [2]:
mutable struct TriangleNode
    orientation::Int # -1 means counter clockwise, 1 means clockwise, 0 means unmarked/unoriented
    vertices::SVector{3, Int} # In lexicographical order. Also the order corresponding to index 0
    neighbors::Union{Vector{TriangleNode}, Nothing} # An array storing pointers to the neighbors.
end

You might notice that we do not store a vector of pointers to the `fnext` triangles (see lecture). Instead we will compute them whenever we need them. This will likely be slower when checking for orientability. However this keeps the `TriangleNode` a bit more lightweight which is nice.

A second observation that is useful to make is that the type declared for neighbors is 

`Union{Vector{TriangleNode}, Nothing}`, which means: 

* **EITHER** a vector of triangles 
* **OR** `nothing`

We declare it like this because when first constructing the node, we will not know its neighbors. Therefor we initialize this array as `nothing` until we constructed all nodes and found all neighbor relations. Then we will set the completed neighbor vector.

Check the output of the following cell to see `nothing`. 😺

In [3]:
nothing

## Some assumptions and conventions

Firstly we will label vertices $1, ... , n$. Triangles are represented as collections of vertex labels. Meaning $[1,2,3]$, $[1,2,4]$ and so on. In this way we avoid a map from some `Char`s $a,b,c$ to some `Int`s $1,2,3$.

Secondly we will asume, that the `zeroth_order` (meaning $\iota = 0$) of some triangle $\mu$ is the lexicographical order of its vertices. This means some triangle with vertices $1$, $2$ and $3$ will have zeroth order $[1,2,3]$. 

We will refer to `TriangleNode`s as $\mu$ and to **ordered triangles** by a pair $(\mu, \iota)$.

Consider the following array of triangle information representing a tetrahedron. Every face is ordered and you can assume that each data set you will get to create your triangulation graph is of this form.

In [16]:
tetrahedron_triangulation = [@SVector[1,2,3], @SVector[1,2,4], @SVector[1,3,4], @SVector[2,3,4]]

4-element Vector{SVector{3, Int64}}:
 [1, 2, 3]
 [1, 2, 4]
 [1, 3, 4]
 [2, 3, 4]

The following is a constructor of our `TriangleNode` that takes a `SVector` of length $3$ as Input and constructs a `TriangleNode` with unitialized neighbors and orientation set to $0$ meaning *unoriented*. 

It also makes sure that the Input is lexicographically ordered.

In [17]:
function TriangleNode(vertices::SVector{3, Int})
    if !(vertices[1] < vertices[2] < vertices[3])
        error("Vertices not sorted!")
    end
    
    TriangleNode(0, vertices, nothing)
end

TriangleNode

In [18]:
# Example of constructor usage
TriangleNode(@SVector[1,2,3])

TriangleNode(0, [1, 2, 3], nothing)

# Graph initalization

The first thing you need to do is to write a function that takes a `Vector{SVector{3, Int64}}`, representing ordered vertex sets of triangles, as Input and creates a graph of `TriangleNodes`. 

**All nodes in this graph need to have initialized neighbors!**

The function should return either an `Iterator` over the nodes of the triangle graph or a `Vector` containing them.

In [19]:
# Reminder on iterators: 
iter = 1:3 # This is an iterator

1:3

In [20]:
elements = collect(iter) # This is the data the iterator iterates over.

3-element Vector{Int64}:
 1
 2
 3

Tips for implementing this: 

* Construct a dicitonary containing the `faces` as keys and the constructed `TriangleNodes` as values
* Construct neighbor vectors for each `TriangleNode` in the dictionary and assign them. *Hint: You can also use the keys to check if two triangles share an edge*.
* return `values(your_dict)` which already is an iterator.


In [21]:
# Example input: 
# tetrahedron_triangulation = [@SVector[1,2,3], @SVector[1,2,4], @SVector[1,3,4], @SVector[2,3,4]]
function construct_graph(faces::Vector{SVector{3, Int64}})
    # Create a an OrderedTriangle for each face of the input and save it in a dict.
    # We use the SVector representing the face as key. We still need to include the neighbor relations.
    node_dict = 
    Dict((first_order => TriangleNode(first_order) for first_order in faces))
    
    for current in faces
        neighbors = []  
        for pot_neighbor in faces
            if current != pot_neighbor
                
                edge_one = current[SVector(1,2)]
                if issubset(edge_one, pot_neighbor)
                    push!(neighbors, node_dict[pot_neighbor])
                end
                
                edge_two = current[SVector(1,3)]
                if issubset(edge_two, pot_neighbor)
                    push!(neighbors, node_dict[pot_neighbor])
                end
                
                edge_three = current[SVector(2,3)]
                if issubset(edge_three, pot_neighbor)
                    push!(neighbors, node_dict[pot_neighbor])
                end
            end
        end
        node_dict[current].neighbors = neighbors
    end
    values(node_dict) #Returns an iterator over all ordered triangles
end

construct_graph (generic function with 1 method)

**You can test your implementation by running the following cell.**

In [22]:
tetrahedron_triangulation = [@SVector[1,2,3], @SVector[1,2,4], @SVector[1,3,4], @SVector[2,3,4]]
graph = construct_graph(tetrahedron_triangulation)

# Check that every triangle has three neighbors
for μ in graph
    @assert length(μ.neighbors) == 3
end

# Make sure all neighbors share two vertices
for μ in graph
    for neighbor in μ.neighbors
        @assert count(i->(i in μ.vertices), neighbor.vertices) == 2
    end
end

# Make sure that every triangle is its own neighbors neighbor!
for μ in graph
    for neighbor in μ.neighbors
        @assert μ in neighbor.neighbors
    end
end

# ORDER!

Write a function called `get_order(zeroth_order::SVector{3, Int}, ι::Int)` that takes some `zeroth_order` and an order index `iota` as input and returns an `SVector{3, Int64}` representing the order of vertices corresponding  to `iota`.

Check the tests for further insights!

In [23]:
function get_order(zeroth_order::SVector{3, Int}, ι::Int)
    if ι == 0 return @SVector[zeroth_order[1], zeroth_order[2], zeroth_order[3]] end 
    if ι == 1 return @SVector[zeroth_order[2], zeroth_order[3], zeroth_order[1]] end
    if ι == 2 return @SVector[zeroth_order[3], zeroth_order[1], zeroth_order[2]] end
    if ι == 4 return @SVector[zeroth_order[2], zeroth_order[1], zeroth_order[3]] end
    if ι == 5 return @SVector[zeroth_order[3], zeroth_order[2], zeroth_order[1]] end
    if ι == 6 return @SVector[zeroth_order[1], zeroth_order[3], zeroth_order[2]] end
    @SVector[nothing]
end

# Testing
zeroth_order = @SVector[1,2,3]
@assert get_order(zeroth_order, 0) == [1,2,3] # NOTE: The return of get_order is an SVector but "==" works as if it were a Base.Vector
@assert get_order(zeroth_order, 1) == [2,3,1]
@assert get_order(zeroth_order, 2) == [3,1,2]
@assert get_order(zeroth_order, 4) == [2,1,3]
@assert get_order(zeroth_order, 5) == [3,2,1]
@assert get_order(zeroth_order, 6) == [1,3,2]
@assert get_order(zeroth_order, 7) == [nothing]

Now write an inverse function `get_iota(order::SVector{3, Int})` that takes an `SVector{3, Int64}` as Input and returns an `Int` representing the order. This is possible because we have agreed that the `zeroth_order` is always the vertex set in lexicographical order.

In [24]:
function get_iota(order::SVector{3, Int})
    zeroth_order = sort(order)
    if order == [zeroth_order[1], zeroth_order[2], zeroth_order[3]] return 0 end
    if order == [zeroth_order[2], zeroth_order[3], zeroth_order[1]] return 1 end
    if order == [zeroth_order[3], zeroth_order[1], zeroth_order[2]] return 2 end
    if order == [zeroth_order[2], zeroth_order[1], zeroth_order[3]] return 4 end
    if order == [zeroth_order[3], zeroth_order[2], zeroth_order[1]] return 5 end
    if order == [zeroth_order[1], zeroth_order[3], zeroth_order[2]] return 6 end
    return 0
end

@assert get_iota(@SVector[1,2,3]) == 0
@assert get_iota(@SVector[2,3,1]) == 1
@assert get_iota(@SVector[3,1,2]) == 2
@assert get_iota(@SVector[2,1,3]) == 4
@assert get_iota(@SVector[3,2,1]) == 5
@assert get_iota(@SVector[1,3,2]) == 6

**Great!** Now write another version `get_order((μ, ι)::Tuple{TriangleNode, Int64})` that takes an *ordered triangle*, i.e. a `TriangleNode`and an order index `iota` as Input.  

Tip: Use the function you already wrote! The data you need is contained in the `TriangleNode`.

In [25]:
function get_order((μ, ι)::Tuple{TriangleNode, Int64})
    get_order(μ.vertices, ι)
end

# Testing

μ = TriangleNode(@SVector[1,2,3])
@assert get_order((μ, 0)) == [1,2,3]
@assert get_order((μ, 1)) == [2,3,1]
@assert get_order((μ, 2)) == [3,1,2]
@assert get_order((μ, 4)) == [2,1,3]
@assert get_order((μ, 5)) == [3,2,1]
@assert get_order((μ, 6)) == [1,3,2]
@assert get_order((μ, 7)) == [nothing]

Implement 

* `enext((μ, ι)::Tuple{TriangleNode, Int64})` 
* `sym((μ, ι)::Tuple{TriangleNode, Int64})`

from the lecture. Note that although we do input a tuple containing a `TriangleNode` because we like to call some composites of functions later on strictly speaking we only need $\iota$ as input. 

The function should return a tuple $(\mu, \kappa)$ where $\kappa$ is the new order index.

In [26]:
# 0 -> 1 -> 2 -> 0 | 4 -> 6 -> 5 -> 4
function enext((μ, ι)::Tuple{TriangleNode, Int64})
    @assert ι in 0:2 || ι in 4:6
    if 0 <= ι <= 2 
        κ = (ι + 1) % 3
    elseif 4 <= ι <= 6
        κ = (ι + 1) % 3 +4
    end
    (μ,κ)
end

# 0 <-> 4 | 1 <-> 5 | 2 <-> 6
function sym((μ, ι)::Tuple{TriangleNode, Int64})
    @assert ι in 0:2 || ι in 4:6
    κ = (ι + 4) % 8
    (μ,κ)
end

sym (generic function with 1 method)

In [27]:
# Testing

μ = TriangleNode(@SVector[1,2,3])
ι = 0

@assert μ.orientation == 0
(μ, κ) = enext((μ, ι))
@assert ι == 0
@assert κ == 1
@assert enext(enext((μ, 1))) == (μ, 0)

@assert enext(enext(enext((μ, 6)))) == (μ, 6)
@assert sym(enext((μ, 6))) == (μ, 1)
@assert sym(sym((μ, 1))) == (μ, 1)

Alright now we still need `fnext`. Unlike described in the lecture we will not compute it once and save all references in the `TriangleNode` structs. We will compute `fnext` whenever we need it. In terms of implementation this just means, that `fnext`contains code we would have otherwise called on initialization.

A useful helper function for that is `get_lead_edge((μ, ι)::Tuple{OrderedTriangle, Int64})` which you can implement and test in the next cell. 

**Examples**: 
+ An ordered triangle $[1,2,3]$ (which means $\iota = 0$) has lead edge $[1,2]$
+ An ordered triangle $[2,1,3]$ (which means $\iota = 4$) has lead edge $[2,1]$
+ You can also get these examples from the tests :)

In [28]:
function get_lead_edge((μ, ι)::Tuple{TriangleNode, Int64})
    get_order((μ, ι))[SVector(1,2)]
end


# Testing
μ = TriangleNode(@SVector[1,2,3])
ι = 0

@assert get_lead_edge((μ, ι)) == [1,2]
@assert get_lead_edge(sym((μ, ι))) == [2,1]

Now we can write `fnext((μ, ι)::Tuple{OrderedTriangle, Int64})` which returns the ordered triangle $(\nu, \kappa)$ that has the same lead edge as $(\mu, \iota)$.

In [29]:
function fnext((μ, ι)::Tuple{TriangleNode, Int64})
    lead_edge = get_lead_edge((μ, ι))
    for neighbor in μ.neighbors
        if issubset(lead_edge, neighbor.vertices)
             last_vertex = [x for x ∈ neighbor.vertices if x ∉ lead_edge][1]
             fnext_order = @SVector[lead_edge[1], lead_edge[2], last_vertex]
             κ = get_iota(fnext_order)
             return (neighbor, κ)
        end
    end
end

fnext (generic function with 1 method)

In [30]:
# Testing
tetrahedron_triangulation = [@SVector[1,2,3], @SVector[1,2,4], @SVector[1,3,4], @SVector[2,3,4]]
graph = construct_graph(tetrahedron_triangulation)

μ = first(graph)
ι = 0
(ν, κ) = fnext((μ, ι))

@assert get_lead_edge((μ, ι)) == get_lead_edge((ν, κ))
@assert ν in μ.neighbors
@assert fnext((ν, κ)) == (μ, ι)

Finally implement the `is_orientable((μ, ι)::Tuple{TriangleNode, Int64})` function. You can use the `orientation` field from the `TriangleNode` struct for saving the orientation and maintaining if a triangle was already marked.

In [34]:
function is_orientable((μ, ι)::Tuple{TriangleNode, Int64})
    if μ.orientation == 0
        0 <= ι <= 2 ? μ.orientation = -1 : μ.orientation = 1 
        a = is_orientable(fnext(sym((μ, ι))))
        b = is_orientable(fnext(enext(sym((μ, ι)))))
        c = is_orientable(fnext(enext(enext(sym((μ, ι))))))
        return a && b && c
    else
        return (μ.orientation == -1 && ι ∈ 0:2) || (μ.orientation == 1 && ι ∈ 3:6)
    end
end

is_orientable (generic function with 1 method)

In [35]:
tetrahedron_triangulation = [@SVector[1,2,3], @SVector[1,2,4], @SVector[1,3,4], @SVector[2,3,4]]
graph = construct_graph(tetrahedron_triangulation)

μ = first(graph)
@assert is_orientable((μ, 0)) == true

In [36]:
pp_triangulation = [@SVector[1,2,4], @SVector[1,3,4], @SVector[1,2,6], @SVector[1,5,6], @SVector[1,3,5], @SVector[2,3,5], @SVector[2,4,5], @SVector[4,5,6], @SVector[3,4,6], @SVector[2,3,6]]
graph = construct_graph(pp_triangulation)

μ = first(graph)
@assert is_orientable((μ, 0)) == false

# Congratulations!

Given the input as we had in this exercise, it is indeed trivial to get the number of triangles. Therefor, as soon as we have figured out how to check orientability, calculating the genus is too!